# Character Level Language Model Using  Bigram and Probabilities

In [3]:
# Import Libraries
import torch

In [ ]:
# Get the dataset
!wget "https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt"

In [ ]:
#Reading training data
words =  open('/content/names.txt','r').read().splitlines()

In [ ]:
# Total Words
len(words)
# Length of Word with minimum character
min(len(word) for word in words)
# Length of Word with maximum character
max(len(word) for word in words)


In [ ]:
# Adding start and stop character to sequence of words
b = {}
for word in words:
  chs = ['<s>'] + list(word) + ['<e>']
  for ch1,ch2 in zip(chs,chs[1:]):
    bigram = (ch1,ch2)
    b[bigram] = b.get(bigram,0) + 1

In [ ]:
sorted(b.items(), key = lambda kv: -kv[1])

In [ ]:
#creating array
N= torch.zeros((28,28), dtype = torch.int32)

In [ ]:
# Adding special charcters index to hash map
chars = sorted(list(set(''.join(words))))
stoi = {s:i for i,s in enumerate(chars)}
stoi['<s>'] = 26
stoi['<e>'] = 27

In [ ]:
#creating array
N= torch.zeros((28,28), dtype = torch.int32)

for word in words:
  chs = ['<s>'] + list(word) + ['<e>']
  for ch1,ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1,ix2] += 1

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(N)

In [ ]:
itos = {i:s for i,s in enumerate(stoi.items())}

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(50,50))
plt.imshow(N, cmap='Blues')
for i in range(28):
    for j in range(28):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

In [ ]:
# Sampling example
p = N[0, :].float()
p = p/p.sum()

In [ ]:
g = torch.Generator().manual_seed(2147483647)
# p = torch.rand(3,generator=g)
p = p/p.sum()

In [ ]:

torch.multinomial(p,num_samples=1, replacement=True, generator=g )
# to get the index we will use .item()
ix = torch.multinomial(p,num_samples=1, replacement=True, generator=g ).item()
ix
itos[ix]

In [ ]:
P = N.float()
P /= P.sum(0,keepdims=True)

In [ ]:
 P = (N+10).float()

 # keepdims= False will mess up the predictions it is very important to study broadcasting rules
 # in pytorch
 P /= P.sum(1, keepdims=True)

In [ ]:
 # Sampling 
g = torch.Generator().manual_seed(2147483647)
ix = 0
for i in range(20):
  out = []
  while True:
    # p = N[ix].float()
    # p = p/p.sum()
    # # p = torch.ones(27)/27.0
    p = P[ix]
    ix = torch.multinomial(p,num_samples=1, replacement=True, generator=g ).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

In [ ]:
# Negative Likelihood
negativeloglikelihood = 0
count = 0
for w in ['bvend']:
  chs = '.' + w + '.'
  for ch1, ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1,ix2]
    logprob = torch.log(prob)
    negativeloglikelihood += logprob
    count += 1

print(f'Log Likelihood is {-negativeloglikelihood.item()/count: .4f}')